In [ ]:
#Set up stuff from notebook
import os
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px
from jaxtyping import Float
from functools import partial
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

DEVELOPMENT_MODE = False
# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2

IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"
print("works")

# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh

import plotly.io as pio

pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Neel")

torch.set_grad_enabled(False)

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

device = utils.get_device()

works
Using renderer: notebook_connected


In [221]:
def get_params(model, show_param = False):
    count = 0
    parameters = []
    names = []
    for name, param in model.named_parameters():
        if show_param == True:
            print(name, param.size())
        names.append([name, param.size()])
        parameters.append(param.data)
        count += 1
    print(f"Model has {count} named parameters")
    return parameters, names

def unpack_names(names, first = 0, last = None):
    if last == None:
        for item in names[first:len(names)-1]:
            print(f"Name of param: {item[0]:<30} Dimensions of tensor: {item[1]:<30}")
    else:
        for item in names[first: last]:
            print(f"Name of param: {item[0]:<30} Dimensions of tensor: {str(item[1]):<30}")

def compare(model_1, model_2):
    different = []
    model_1_params, model_1_names = get_params(model_1)
    model_2_params, model_2_names = get_params(model_2)

    if len(model_1_params) != len(model_2_params):
        print("There are a different number of parameters between the models. Please check model config")
        print(f"Model 1 has {len(model_1_params)} parameters. Model 2 has {len(model_2_params)}")

    for i in range(min(len(model_1_params),(len(model_2_params)))):
        if model_1_names[i][1] != model_2_names[i][1]:
            different.append([model_1_names[i], model_2_names[i], ("index", i)])

    print(f"There are {len(different)} parameters that are different from each other")
    return different 


In [200]:
hooked_model = HookedTransformer.from_pretrained("gpt2", device=device)

Loaded pretrained model gpt2 into HookedTransformer


In [201]:
hooked_params, hooked_names = get_params(hooked_model)

Model has 148 named parameters


In [202]:
unpack_names(hooked_names, first = 0, last = 10)

Name of param: embed.W_E                      Dimensions of tensor: torch.Size([50257, 768])      
Name of param: pos_embed.W_pos                Dimensions of tensor: torch.Size([1024, 768])       
Name of param: blocks.0.attn.W_Q              Dimensions of tensor: torch.Size([12, 768, 64])     
Name of param: blocks.0.attn.W_O              Dimensions of tensor: torch.Size([12, 64, 768])     
Name of param: blocks.0.attn.b_Q              Dimensions of tensor: torch.Size([12, 64])          
Name of param: blocks.0.attn.b_O              Dimensions of tensor: torch.Size([768])             
Name of param: blocks.0.attn.W_K              Dimensions of tensor: torch.Size([12, 768, 64])     
Name of param: blocks.0.attn.W_V              Dimensions of tensor: torch.Size([12, 768, 64])     
Name of param: blocks.0.attn.b_K              Dimensions of tensor: torch.Size([12, 64])          
Name of param: blocks.0.attn.b_V              Dimensions of tensor: torch.Size([12, 64])          


In [222]:
hooked_param, hooked_names = get_params(hooked_model)

Model has 148 named parameters


In [203]:
from transformers import GPT2Model

gpt2 = GPT2Model.from_pretrained('gpt2')


In [204]:
gpt2_params, gpt2_names = get_params(gpt2)

Model has 148 named parameters


In [205]:
gpt2_unpacked = unpack_names(gpt2_names, first = 0, last = 10)

Name of param: wte.weight                     Dimensions of tensor: torch.Size([50257, 768])      
Name of param: wpe.weight                     Dimensions of tensor: torch.Size([1024, 768])       
Name of param: h.0.ln_1.weight                Dimensions of tensor: torch.Size([768])             
Name of param: h.0.ln_1.bias                  Dimensions of tensor: torch.Size([768])             
Name of param: h.0.attn.c_attn.weight         Dimensions of tensor: torch.Size([768, 2304])       
Name of param: h.0.attn.c_attn.bias           Dimensions of tensor: torch.Size([2304])            
Name of param: h.0.attn.c_proj.weight         Dimensions of tensor: torch.Size([768, 768])        
Name of param: h.0.attn.c_proj.bias           Dimensions of tensor: torch.Size([768])             
Name of param: h.0.ln_2.weight                Dimensions of tensor: torch.Size([768])             
Name of param: h.0.ln_2.bias                  Dimensions of tensor: torch.Size([768])             


In [223]:
diff = compare(hooked_model, gpt2)

diff[0:5]

Model has 148 named parameters
Model has 148 named parameters
There are 98 parameters that are different from each other


[[['blocks.0.attn.W_Q', torch.Size([12, 768, 64])],
  ['h.0.ln_1.weight', torch.Size([768])],
  ('index', 2)],
 [['blocks.0.attn.W_O', torch.Size([12, 64, 768])],
  ['h.0.ln_1.bias', torch.Size([768])],
  ('index', 3)],
 [['blocks.0.attn.b_Q', torch.Size([12, 64])],
  ['h.0.attn.c_attn.weight', torch.Size([768, 2304])],
  ('index', 4)],
 [['blocks.0.attn.b_O', torch.Size([768])],
  ['h.0.attn.c_attn.bias', torch.Size([2304])],
  ('index', 5)],
 [['blocks.0.attn.W_K', torch.Size([12, 768, 64])],
  ['h.0.attn.c_proj.weight', torch.Size([768, 768])],
  ('index', 6)]]

In [208]:
hooked_model.blocks[0].attn.W_Q.data

tensor([[[-0.1026, -0.0609, -0.0249,  ...,  0.0203,  0.0658,  0.0478],
         [ 0.0191,  0.0242,  0.0404,  ..., -0.0670,  0.0618, -0.0190],
         [ 0.0038,  0.0081,  0.0532,  ..., -0.0380,  0.0209,  0.0367],
         ...,
         [-0.0461, -0.0057,  0.0348,  ...,  0.0152, -0.0104,  0.0085],
         [ 0.0309,  0.0370,  0.0160,  ..., -0.0175, -0.0892, -0.0513],
         [-0.0747, -0.0391, -0.0486,  ..., -0.0675, -0.0310, -0.0416]],

        [[-0.0146,  0.0116, -0.0336,  ..., -0.0270, -0.0309, -0.0234],
         [-0.0148,  0.0689,  0.0947,  ...,  0.0179,  0.0131, -0.0445],
         [ 0.0237,  0.0168, -0.0566,  ..., -0.0101,  0.0308,  0.0022],
         ...,
         [ 0.0526,  0.0214, -0.0353,  ..., -0.0663, -0.0429,  0.1076],
         [-0.0119,  0.0048, -0.1068,  ...,  0.0063, -0.0678, -0.0890],
         [-0.1548,  0.0178, -0.0272,  ..., -0.0034,  0.0690, -0.0079]],

        [[ 0.0057,  0.0382,  0.0539,  ...,  0.0343,  0.1000, -0.0263],
         [ 0.0242,  0.0208, -0.0582,  ..., -0

In [209]:
gpt2.h[0].ln_1.weight

Parameter containing:
tensor([0.2232, 0.1820, 0.1534, 0.1917, 0.2036, 0.1948, 0.1467, 0.1865, 0.2143,
        0.1956, 0.2118, 0.2153, 0.1882, 0.2074, 0.1871, 0.2040, 0.2044, 0.1900,
        0.1952, 0.0475, 0.1909, 0.2115, 0.1971, 0.2202, 0.1998, 0.2108, 0.2303,
        0.1879, 0.1939, 0.2018, 0.1891, 0.1861, 0.1958, 0.1832, 0.1978, 0.2243,
        0.0706, 0.1958, 0.1943, 0.1939, 0.1978, 0.1951, 0.1995, 0.1912, 0.2083,
        0.2037, 0.1849, 0.1945, 0.2189, 0.0419, 0.1977, 0.1979, 0.0608, 0.1824,
        0.2055, 0.0476, 0.1892, 0.2079, 0.2047, 0.2233, 0.2097, 0.2075, 0.2076,
        0.1793, 0.1312, 0.1841, 0.1939, 0.1561, 0.0577, 0.1948, 0.2048, 0.1717,
        0.1942, 0.1708, 0.1989, 0.1993, 0.2082, 0.1071, 0.1968, 0.1770, 0.2164,
        0.1864, 0.1938, 0.2184, 0.1343, 0.1707, 0.0683, 0.1401, 0.1823, 0.2045,
        0.2007, 0.1853, 0.1783, 0.1889, 0.1870, 0.1975, 0.2114, 0.2108, 0.2083,
        0.2409, 0.1938, 0.2022, 0.0857, 0.1823, 0.1879, 0.1979, 0.1850, 0.1029,
        0.1762, 0.

In [210]:
hooked_model.cfg

HookedTransformerConfig:
{'NTK_by_parts_factor': 8.0,
 'NTK_by_parts_high_freq_factor': 4.0,
 'NTK_by_parts_low_freq_factor': 1.0,
 'act_fn': 'gelu_new',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_scale': 8.0,
 'attn_scores_soft_cap': -1.0,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 64,
 'd_mlp': 3072,
 'd_model': 768,
 'd_vocab': 50257,
 'd_vocab_out': 50257,
 'decoder_start_token_id': None,
 'default_prepend_bos': True,
 'device': device(type='cuda'),
 'dtype': torch.float32,
 'eps': 1e-05,
 'experts_per_token': None,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.02886751345948129,
 'load_in_4bit': False,
 'model_name': 'gpt2',
 'n_ctx': 1024,
 'n_devices': 1,
 'n_heads': 12,
 'n_key_value_heads': None,
 'n_layers': 12,
 'n_params': 84934656,
 'normalization_type': 'LNPre',
 'num_experts': None,
 'original

In [211]:
gpt2.config

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 50257
}

In [ ]:
MODELPATH = r"C:\Users\allan\ResearchStuff\checkpoint-1953"
from transformers import AutoTokenizer, GPTJForCausalLM

gptj = GPTJForCausalLM.from_pretrained(MODELPATH)
gptj_param, gptj_names = get_params(gptj)


unpack_names(gptj_names, first = 0, last = 10)

Model has 125 named parameters
Name of param: transformer.wte.weight         Dimensions of tensor: torch.Size([50257, 768])      
Name of param: transformer.h.0.ln_1.weight    Dimensions of tensor: torch.Size([768])             
Name of param: transformer.h.0.ln_1.bias      Dimensions of tensor: torch.Size([768])             
Name of param: transformer.h.0.attn.k_proj.weight Dimensions of tensor: torch.Size([768, 768])        
Name of param: transformer.h.0.attn.v_proj.weight Dimensions of tensor: torch.Size([768, 768])        
Name of param: transformer.h.0.attn.q_proj.weight Dimensions of tensor: torch.Size([768, 768])        
Name of param: transformer.h.0.attn.out_proj.weight Dimensions of tensor: torch.Size([768, 768])        
Name of param: transformer.h.0.mlp.fc_in.weight Dimensions of tensor: torch.Size([3072, 768])       
Name of param: transformer.h.0.mlp.fc_in.bias Dimensions of tensor: torch.Size([3072])            
Name of param: transformer.h.0.mlp.fc_out.weight Dimension

In [224]:
compare(gpt2, gptj)

Model has 148 named parameters
Model has 125 named parameters
There are a different number of parameters between the models. Please check model config
Model 1 has 148 parameters. Model 2 has 125
There are 103 parameters that are different from each other


[[['wpe.weight', torch.Size([1024, 768])],
  ['transformer.h.0.ln_1.weight', torch.Size([768])],
  ('index', 1)],
 [['h.0.ln_1.bias', torch.Size([768])],
  ['transformer.h.0.attn.k_proj.weight', torch.Size([768, 768])],
  ('index', 3)],
 [['h.0.attn.c_attn.weight', torch.Size([768, 2304])],
  ['transformer.h.0.attn.v_proj.weight', torch.Size([768, 768])],
  ('index', 4)],
 [['h.0.attn.c_attn.bias', torch.Size([2304])],
  ['transformer.h.0.attn.q_proj.weight', torch.Size([768, 768])],
  ('index', 5)],
 [['h.0.attn.c_proj.bias', torch.Size([768])],
  ['transformer.h.0.mlp.fc_in.weight', torch.Size([3072, 768])],
  ('index', 7)],
 [['h.0.ln_2.weight', torch.Size([768])],
  ['transformer.h.0.mlp.fc_in.bias', torch.Size([3072])],
  ('index', 8)],
 [['h.0.ln_2.bias', torch.Size([768])],
  ['transformer.h.0.mlp.fc_out.weight', torch.Size([768, 3072])],
  ('index', 9)],
 [['h.0.mlp.c_fc.weight', torch.Size([768, 3072])],
  ['transformer.h.0.mlp.fc_out.bias', torch.Size([768])],
  ('index', 10)

In [214]:
gptj.config

GPTJConfig {
  "_name_or_path": "C:\\Users\\allan\\ResearchStuff\\checkpoint-1953",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTJForCausalLM"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gptj",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.0,
  "rotary_dim": 64,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 50257
}